In [160]:
source_layer = "ra"
source_data = "Day_Ahead_Half_Hourly_Demand_Forecast_Performance"
source_container = "sink"
source_account = "dapffuksrawdl01"
source_file_path = "demand_forecasting/Day_Ahead_Half_Hourly_Demand_Forecast_Performance/CSV/2023/09/1B-Incentive_202309151200_202309151101.csv"
#Printing the path
path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (source_container, source_account, source_file_path)
print(path)

StatementMeta(aspool, 241176, 201, Finished, Available)

abfss://sink@dapffuksrawdl01.dfs.core.windows.net/demand_forecasting/Day_Ahead_Half_Hourly_Demand_Forecast_Performance/CSV/2023/09/1B-Incentive_202309151200_202309151101.csv


In [161]:
#Loading the file into a dataframe
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw = spark.read.load('abfss://%s@%s.dfs.core.windows.net/%s' % (source_container, source_account, source_file_path), format='csv',Header= 'True')
print(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw.count())
display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw)

StatementMeta(aspool, 241176, 202, Finished, Available)

42382


SynapseWidget(Synapse.DataFrame, 6b438a92-1d7d-4359-852c-deedf78c0e4e)

In [162]:
from datetime import datetime
import pyspark.sql.types as T
import pyspark.sql.functions as F

def user_defined_timestamp(date_col):
    _date = datetime.strptime(date_col, '%Y-%m-%d')
    return _date.strftime('%d%m%Y')
user_defined_timestamp_udf = F.udf(user_defined_timestamp, T.StringType())
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw1 = Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw.withColumn('CUSTOM_ID_DATE', user_defined_timestamp_udf('DATE'))
from pyspark.sql.functions import concat_ws
##write query for concatenate customi'd
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc1=Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw1.withColumn("CUSTOM_ID", concat_ws("_","SETTLEMENT_PERIOD","DEMAND_FORECAST","CUSTOM_ID_DATE"))
display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc1)
print(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc1.count())

StatementMeta(aspool, 241176, 203, Finished, Available)

SynapseWidget(Synapse.DataFrame, aac86768-709b-48df-a833-069131e5d995)

42382


In [163]:
#Dropping "CUSTOM_ID_SETTLEMENT_DATE" columns as they are externally added in Raw layer
cl_cols_to_drop = ["CUSTOM_ID_DATE"]
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc = Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc1.drop(*cl_cols_to_drop)
#checking the RAW Layer data to see whether the column are dropped or not
display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc)

StatementMeta(aspool, 241176, 204, Finished, Available)

SynapseWidget(Synapse.DataFrame, be832567-9355-4b3f-aa1f-43de85bf6acf)

In [164]:
#List of parameters
target_layer = "cl"
target_source = "day_ahead_halfhourly_demand_forecast"
target_container = "datatransformation"
target_account = "dapffukscleanseddl01"
target_file_path = "energy_balance/day_ahead_halfhourly_demand_forecast/v1_deltalake"

StatementMeta(aspool, 241176, 205, Finished, Available)

In [165]:
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed = spark.read.load('abfss://%s@%s.dfs.core.windows.net/%s/' % (target_container, target_account, target_file_path), format='delta')
print(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed.count())
display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed)

StatementMeta(aspool, 241176, 206, Finished, Available)

42391


SynapseWidget(Synapse.DataFrame, f0417272-6c20-4ebf-98a0-c9a478d75259)

In [166]:
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed.printSchema()
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc.printSchema()

StatementMeta(aspool, 241176, 207, Finished, Available)

root
 |-- MONTH: string (nullable = false)
 |-- DATE: date (nullable = false)
 |-- DATETIME: timestamp (nullable = false)
 |-- SETTLEMENT_PERIOD: integer (nullable = false)
 |-- DEMAND_FORECAST: integer (nullable = true)
 |-- DEMAND_OUTTURN: integer (nullable = true)
 |-- TRIAD_AVOIDANCE_ESTIMATE: integer (nullable = true)
 |-- TRIAD_AVOIDANCE_CORRECTED_DEMAND_OUTTURN: integer (nullable = true)
 |-- APE: float (nullable = true)
 |-- ABSOLUTE_ERROR: integer (nullable = true)
 |-- PUBLISH_DATETIME: timestamp (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)
 |-- _BATCHID: string (nullable = true)
 |-- _SRCFILENAME: string (nullable = true)
 |-- _CREATEDON: timestamp (nullable = true)
 |-- _UPDATEDON: timestamp (nullable = true)

root
 |-- Month: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Settlement_Period: string (nullable = true)
 |-- Demand_Forecast: string (nullable = true)
 |-- Demand_Outturn: string (nullable =

In [167]:
%run /Tools/Utilities

StatementMeta(, 241176, -1, Finished, Available)

..........Running Utilities Notebook.........


In [168]:
display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed.groupBy('_SRCFILENAME').count().where(col('count')>1))

StatementMeta(aspool, 241176, 219, Finished, Available)

SynapseWidget(Synapse.DataFrame, 716a103e-88dd-49fe-9458-94531ea36b72)

In [169]:
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_re=Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed.filter(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed._SRCFILENAME=='/sink/demand_forecasting/Day_Ahead_Half_Hourly_Demand_Forecast_Performance/CSV/2023/09/1B-Incentive_202309151200_202309151101.csv?version=1694775796830?flength=4013321')
print(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_re.count())


StatementMeta(aspool, 241176, 220, Finished, Available)

42382


In [170]:
print(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_re.count())
print(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_re.dropDuplicates(["CUSTOM_ID"]).count())

StatementMeta(aspool, 241176, 221, Finished, Available)

42382
42382


In [171]:
#Dropping some of the columns as they are externally added in Cleansed layer
cl_cols_to_drop = ["_BATCHID", "_SRCFILENAME", "_UPDATEDON", "_CREATEDON"]
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_reconc = Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_re.drop(*cl_cols_to_drop)
#checking the RAW Layer data to see whether the column are dropped or not
display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_reconc)


StatementMeta(aspool, 241176, 222, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2f62817c-016b-4477-88ed-e49c2b5a1f1f)

In [172]:
# changing the schema of columns from string to respective data types as mentioned in cleansed layer
#Date
#Datetime
#Settlement_Period
#Demand_Forecast
#Demand_Outturn
#TRIAD_Avoidance_Estimate
#TRIAD_Avoidance_Corrected_Demand_Outturn
#APE
#Absolute_Error
#Publish_Datetime

from pyspark.sql.types import DateType,IntegerType,TimestampType,StringType,FloatType
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new = Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc \
            .withColumn("Date" ,
              Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc["Date"]
              .cast(DateType()))  \
            .withColumn("Datetime" ,
              Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc["Datetime"]
              .cast(TimestampType()))  \
            .withColumn("Settlement_Period" ,
              Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc["Settlement_Period"]
              .cast(IntegerType()))  \
            .withColumn("Demand_Outturn" ,
              Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc["Demand_Outturn"]
              .cast(IntegerType()))  \
            .withColumn("TRIAD_Avoidance_Estimate" ,
              Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc["TRIAD_Avoidance_Estimate"]
              .cast(IntegerType()))  \
            .withColumn("APE" ,
              Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc["APE"]
              .cast(FloatType()))  \
            .withColumn("Absolute_Error" ,
              Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc["Absolute_Error"]
              .cast(IntegerType()))  \
            .withColumn("Publish_Datetime" ,
              Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc["Publish_Datetime"]
              .cast(TimestampType()))  \
            
            
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_reconc.printSchema() 
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new.printSchema()                             
display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new)
print(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new.count())

StatementMeta(aspool, 241176, 223, Finished, Available)

root
 |-- Month: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Settlement_Period: string (nullable = true)
 |-- Demand_Forecast: string (nullable = true)
 |-- Demand_Outturn: string (nullable = true)
 |-- TRIAD_Avoidance_Estimate: string (nullable = true)
 |-- TRIAD_Avoidance_Corrected_Demand_Outturn: string (nullable = true)
 |-- APE: string (nullable = true)
 |-- Absolute_Error: string (nullable = true)
 |-- Publish_Datetime: string (nullable = true)
 |-- CUSTOM_ID: string (nullable = false)

root
 |-- Month: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Datetime: timestamp (nullable = true)
 |-- Settlement_Period: integer (nullable = true)
 |-- Demand_Forecast: string (nullable = true)
 |-- Demand_Outturn: integer (nullable = true)
 |-- TRIAD_Avoidance_Estimate: integer (nullable = true)
 |-- TRIAD_Avoidance_Corrected_Demand_Outturn: string (nullable = true)
 |-- APE: float (nullable = true)
 |-- Absolute_

SynapseWidget(Synapse.DataFrame, 93616279-97ff-4c34-8515-4d92fe817068)

42382


In [173]:
from pyspark.sql.types import  TimestampType, TimestampType,IntegerType,IntegerType,FloatType
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new1=Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new.withColumnRenamed("Month","MONTH")\
        .withColumnRenamed("Date","DATE")\
        .withColumnRenamed("Datetime","DATETIME")\
        .withColumnRenamed("Settlement_Period","SETTLEMENT_PERIOD")\
        .withColumnRenamed("Demand_Forecast","DEMAND_FORECAST")\
        .withColumnRenamed("Demand_Outturn","DEMAND_OUTTURN")\
        .withColumnRenamed("TRIAD_Avoidance_Estimate","TRIAD_AVOIDANCE_ESTIMATE")\
        .withColumnRenamed("TRIAD_Avoidance_Corrected_Demand_Outturn","TRIAD_AVOIDANCE_CORRECTED_DEMAND_OUTTURN")\
        .withColumnRenamed("APE","APE")\
        .withColumnRenamed("Absolute_Error","ABSOLUTE_ERROR")\
        .withColumnRenamed("Publish_Datetime","PUBLISH_DATETIME")
display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new1)

StatementMeta(aspool, 241176, 224, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5766cd4b-7b2c-4c3e-a44a-2339e3db114d)

In [174]:
#Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_n= Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new1[['CUSTOM_ID','SETTLEMENT_DATE','SETTLEMENT_PERIOD','ND','FORECAST_ACTUAL_INDICATOR','TSD','ENGLAND_WALES_DEMAND','EMBEDDED_WIND_GENERATION','EMBEDDED_WIND_CAPACITY','EMBEDDED_SOLAR_GENERATION','EMBEDDED_SOLAR_CAPACITY','NON_BM_STOR','PUMP_STORAGE_PUMPING','IFA_FLOW','IFA2_FLOW','BRITNED_FLOW','MOYLE_FLOW','EAST_WEST_FLOW','NEMO_FLOW','NSL_FLOW','ELECLINK_FLOW']]
#display(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_n)

StatementMeta(aspool, 241176, 225, Finished, Available)

In [177]:
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_ne= Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_new1.sort("CUSTOM_ID")
Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_reconc1= Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_reconc.sort("CUSTOM_ID")

StatementMeta(aspool, 241176, 228, Finished, Available)

In [178]:
data_reconc = Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_ne.exceptAll(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_reconc1).unionAll(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_cleansed_reconc1.exceptAll(Day_Ahead_Half_Hourly_Demand_Forecast_Performance_raw_ne))
print(data_reconc.count())
display(data_reconc)

StatementMeta(aspool, 241176, 229, Finished, Available)

0


SynapseWidget(Synapse.DataFrame, 9b5c165f-b39b-406e-addb-e9b62e687825)